In [1]:
import googlemaps
import folium
import pandas as pd
import numpy as np
from haversine import haversine #거리구하기
from sklearn.preprocessing import minmax_scale #정규화

In [80]:
total_df = pd.read_csv("./data/total.csv", encoding = "EUC-KR" )
df1 = total_df.rename(index=str, columns={"Unnamed: 0" : "ID"})

# 역삼동 중심 = 역삼역 /  좌표 : 37.500742, 127.036891
to_map = folium.Map(location=[37.500742, 127.036891], zoom_start=18)

## 밝기

In [81]:
from  random import randint
light = []
for i in range(0, len(total_df)) : 
    light.append(randint(0,5))
    
light[0:5]
df1['밝기'] = light
df1.head(10)

,ID,분류,명칭,위도,경도,밝기
0,0,보안등,역삼1동833,37.493349,127.032033,4
1,1,보안등,역삼1동833,37.493349,127.032033,1
2,2,보안등,역삼1동833,37.493349,127.032033,5
3,3,보안등,역삼1동833,37.493349,127.032033,1
4,4,보안등,역삼1동833,37.493349,127.032033,1
5,5,보안등,역삼1동832-25,37.492834,127.031413,4
6,6,보안등,역삼1동832-27,37.492632,127.031508,3
7,7,보안등,역삼1동832-28,37.492480,127.031581,0
8,8,보안등,역삼1동832-28,37.492480,127.031581,4
9,9,보안등,역삼1동832-29,37.492343,127.031644,4


## 거리 

In [83]:
strt = (37.506059, 127.036863) #출발지 37.506059, 127.036863
dstn = (37.509122, 127.043816) #도착지

#총 거리
d_m = haversine(strt, dstn, unit='m') # in meters
d_km = haversine(strt, dstn,  unit='m')  # in kilometers

#block단위 
# 경유지가 총 3개까지만 가능하니까 3개의 반경으로 나누자. 700 이상인 경우는 api다시 받기 (추후에)
dvd = d_m / 4
print( "단위거리(dvd) : ",  dvd, ", 총거리(d_m) : " , d_m)

단위거리(dvd) :  175.38377731330854 , 총거리(d_m) :  701.5351092532342


In [84]:
get_distance = lambda x : haversine(strt, [x['위도'], x['경도']] ,  unit='m') 
df1['거리'] = df1.apply(get_distance , axis = 1) 
df1.head(10)

,ID,분류,명칭,위도,경도,밝기,거리
0,0,보안등,역삼1동833,37.493349,127.032033,4,1476.148279
1,1,보안등,역삼1동833,37.493349,127.032033,1,1476.148279
2,2,보안등,역삼1동833,37.493349,127.032033,5,1476.148279
3,3,보안등,역삼1동833,37.493349,127.032033,1,1476.148279
4,4,보안등,역삼1동833,37.493349,127.032033,1,1476.148279
5,5,보안등,역삼1동832-25,37.492834,127.031413,4,1547.198259
6,6,보안등,역삼1동832-27,37.492632,127.031508,3,1566.001976
7,7,보안등,역삼1동832-28,37.492480,127.031581,0,1580.166885
8,8,보안등,역삼1동832-28,37.492480,127.031581,4,1580.166885
9,9,보안등,역삼1동832-29,37.492343,127.031644,4,1593.104348


## 1. 반경 내에 해당하는 포인트 구하기

In [113]:
def get_points_in_range(df):
    distance = lambda x : haversine(point, [x['위도'], x['경도']] ,  unit='m') 
    
    n = int(0)
    pntInRange = df[df.거리 <= 80].reset_index(drop=True)
    res = pntInRange.sort_values(by = ['거리','밝기'], ascending=[True, False] ).reset_index(drop=True)
    
    return res

points = get_points_in_range(df1)
points

,ID,분류,명칭,위도,경도,밝기,거리
0,1385,보안등,역삼1동 657-30,37.506225,127.036830,1,18.637796
1,1386,보안등,역삼1동 660-21,37.505911,127.036988,3,19.805813
2,1970,CCTV,역삼동 657-27 (참사랑어린이집 앞),37.505911,127.036724,3,20.522308
3,1387,보안등,역삼1동 657-23,37.505983,127.036558,2,28.195673
4,1500,보안등,역삼1동 660-22,37.505799,127.037037,3,32.735255
5,1319,보안등,역삼1동 657-38,37.506325,127.037169,3,40.062620
6,1489,보안등,역삼1동 660-13,37.505778,127.036567,3,40.748673
7,1490,보안등,역삼1동 660-13,37.505778,127.036567,0,40.748673
8,1388,보안등,역삼1동 657-21,37.506264,127.036427,3,44.726531
9,1491,보안등,역삼1동 657-18,37.506080,127.036341,5,46.115954


## 2. 가중치 구하기
* A*(A Start 알고리즘 적용)  
> **F** = G + H  
> G : 출발지로부터의 거리  
> H : 목적지로부터의 거리  


In [98]:
# #1. F 구하기
def get_gf(sp, ep, df ):
    g = np.array(df.apply(lambda x : haversine(sp, [x['위도'], x['경도']] ,  unit='m') , axis = 1 )).reshape(-1,1)
    h = np.array(df.apply(lambda x : haversine(ep, [x['위도'], x['경도']] ,  unit='m') , axis = 1 )).reshape(-1,1)  
    return g+h


* 밝기와 같은 범위로 **scaling** (0~5)

In [99]:
def get_scaled_f(f):
    X_MinMax_scaled = 5 - minmax_scale(f, axis=0, copy=True, feature_range =(0,5))
    return X_MinMax_scaled

In [100]:
points['F'] = get_gf(strt, dstn, points)
points['F_scaled'] =  get_scaled_f(points['F'])
#df1.drop('F',axis=1)
points.head(15)

,ID,분류,명칭,위도,경도,밝기,거리,F,F_scaled
0,1385,보안등,역삼1동 657-30,37.506225,127.036830,1,18.637796,714.037212,4.659664
1,1386,보안등,역삼1동 660-21,37.505911,127.036988,3,19.805813,719.995002,4.462494
2,1970,CCTV,역삼동 657-27 (참사랑어린이집 앞),37.505911,127.036724,3,20.522308,740.816030,3.773431
3,1387,보안등,역삼1동 657-23,37.505983,127.036558,2,28.195673,757.369400,3.225606
4,1500,보안등,역삼1동 660-22,37.505799,127.037037,3,32.735255,735.608008,3.945788
5,1319,보안등,역삼1동 657-38,37.506325,127.037169,3,40.062620,703.753450,5.000000
6,1489,보안등,역삼1동 660-13,37.505778,127.036567,3,40.748673,780.446646,2.461875
7,1490,보안등,역삼1동 660-13,37.505778,127.036567,0,40.748673,780.446646,2.461875
8,1388,보안등,역삼1동 657-21,37.506264,127.036427,3,44.726531,769.835235,2.813055
9,1491,보안등,역삼1동 657-18,37.506080,127.036341,5,46.115954,787.155595,2.239845


* 추가로 밝기를 이용하여 가중치를 적용한다 
> **W**(weight) =  4F + 6L(밝기) 
// 밝기 가중치를 더 높게 조절


In [106]:
points['W'] = points.apply(lambda x : (x['밝기'] * 6 ) + ( x['F_scaled'] * 4) , axis = 1)
points.sort_values('W', ascending = False)

,ID,분류,명칭,위도,경도,밝기,거리,F,F_scaled,W
11,1320,보안등,역삼1동 660-28,37.506019,127.037427,4,49.916634,710.748863,4.768490,43.073961
14,1384,보안등,역삼1동 657-32,37.506563,127.036889,4,56.116535,730.164073,4.125953,40.503812
9,1491,보안등,역삼1동 657-18,37.506080,127.036341,5,46.115954,787.155595,2.239845,38.959382
5,1319,보안등,역삼1동 657-38,37.506325,127.037169,3,40.062620,703.753450,5.000000,38.000000
1,1386,보안등,역삼1동 660-21,37.505911,127.036988,3,19.805813,719.995002,4.462494,35.849974
4,1500,보안등,역삼1동 660-22,37.505799,127.037037,3,32.735255,735.608008,3.945788,33.783154
2,1970,CCTV,역삼동 657-27 (참사랑어린이집 앞),37.505911,127.036724,3,20.522308,740.816030,3.773431,33.093726
25,1834,CCTV,역삼동 655-13,37.506711,127.037181,2,77.736537,721.475218,4.413506,29.654026
17,1502,보안등,역삼1동 660-29,37.505875,127.037490,2,58.972542,723.624189,4.342387,29.369549
8,1388,보안등,역삼1동 657-21,37.506264,127.036427,3,44.726531,769.835235,2.813055,29.252218


# 길찾기 로직 적용